In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment-analysis-1/twitter_training.csv
/kaggle/input/twitter-sentiment-analysis/twitter_training.csv


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import nltk
stemmer = nltk.SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [5]:
def clean(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [6]:
stop_words=stopwords.words('english')
def preprocess_data(text):
    # Clean puntuation, urls, and so o
    # Remove stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    
    return text

In [7]:
df=pd.read_csv('/kaggle/input/twitter-sentiment-analysis-1/twitter_training.csv')
df


,2401,Borderlands,result,hi
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [8]:
df.drop(['2401','Borderlands'],axis=1,inplace=bool(1))

In [9]:
df

,result,hi
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [10]:
l=LabelEncoder()
df['result']=l.fit_transform(df['result'])
df

,result,hi
0,3,I am coming to the borders and I will kill you...
1,3,im getting on borderlands and i will kill you ...
2,3,im coming on borderlands and i will murder you...
3,3,im getting on borderlands 2 and i will murder ...
4,3,im getting into borderlands and i can murder y...
...,...,...
74676,3,Just realized that the Windows partition of my...
74677,3,Just realized that my Mac window partition is ...
74678,3,Just realized the windows partition of my Mac ...
74679,3,Just realized between the windows partition of...


In [11]:
df['hi']=df['hi'].apply(clean)
df

,result,hi
0,3,i am coming to the borders and i will kill you...
1,3,im getting on borderlands and i will kill you all
2,3,im coming on borderlands and i will murder you...
3,3,im getting on borderlands and i will murder y...
4,3,im getting into borderlands and i can murder y...
...,...,...
74676,3,just realized that the windows partition of my...
74677,3,just realized that my mac window partition is ...
74678,3,just realized the windows partition of my mac ...
74679,3,just realized between the windows partition of...


In [12]:
df['hi']=df['hi'].apply(preprocess_data)

In [13]:
df

,result,hi
0,3,come border kill
1,3,im get borderland kill
2,3,im come borderland murder
3,3,im get borderland murder
4,3,im get borderland murder
...,...,...
74676,3,realiz window partit mac like year behind nvi...
74677,3,realiz mac window partit year behind nvidia d...
74678,3,realiz window partit mac year behind nvidia d...
74679,3,realiz window partit mac like year behind nvi...


In [14]:
clf1 = Pipeline([('vectorizer_tri_grams', TfidfVectorizer()),('naive_bayes', (MultinomialNB()))])

In [15]:
xtr,xts,ytr,yts= train_test_split(df['hi'], df['result'], test_size=0.2, random_state=42, stratify=df['result'])

In [16]:
clf1.fit(xtr,ytr)#using navie bayes

Pipeline(steps=[('vectorizer_tri_grams', TfidfVectorizer()),
                ('naive_bayes', MultinomialNB())])

In [17]:
clf1.score(xts,yts)

0.6853451161545157

In [18]:
clf2 = Pipeline([('vectorizer_tri_grams', TfidfVectorizer()),('logistic', (LogisticRegression()))])

In [19]:
clf2.fit(xtr,ytr)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer_tri_grams', TfidfVectorizer()),
                ('logistic', LogisticRegression())])

In [20]:
clf2.score(xts,yts)

0.7285934257213631